In [9]:
#Sytem
import sys,os
import datetime
import numpy as np
import pandas as pd
import warnings
import time
import random
import IPython
warnings.filterwarnings('ignore')

#Visualidation
import sweetviz as sv
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

#EDA
import sweetviz as sv
import sklearn
from sklearn.preprocessing import OneHotEncoder, LabelEncoder



#model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV
from skopt import BayesSearchCV




print('-'*40)

print("Python version: {}". format(sys.version))

print("pandas version: {}". format(pd.__version__))

print("matplotlib version: {}". format(matplotlib.__version__))

print("NumPy version: {}". format(np.__version__))


print("scikit-learn version: {}". format(sklearn.__version__))
print('-'*40)

----------------------------------------
Python version: 3.9.10 (main, Feb 22 2022, 13:54:07) 
[GCC 11.2.0]
pandas version: 1.3.5
matplotlib version: 3.5.1
NumPy version: 1.21.5
scikit-learn version: 1.1.1
----------------------------------------


In [2]:
Target = "Perished"
PATH_LOG = "../logs/log.csv"
ID = 12
SEED = 42

In [3]:
df_train = pd.read_csv('../data/input/train_beta.csv')
df_test = pd.read_csv('../data/input/test_beta.csv')
sample = pd.read_csv('../data/input/gender_submission.csv')
data = pd.concat([df_train,df_test],sort=False)
print('-'*25)
print("Shape of Train Data\n[row :{},column :{}]".format(
    df_train.shape[0], df_train.shape[1]))
print('-'*25)
print("Shape of Test Data\n[row :{},column :{}]".format(
    df_test.shape[0], df_test.shape[1]))
print('-'*25)

-------------------------
Shape of Train Data
[row :891,column :22]
-------------------------
Shape of Test Data
[row :418,column :22]
-------------------------


In [6]:
drop_columns = ["Name","Ticket","Cabin","Embarked","Title",'FareBin',"Fare_Code","Title_Code","Family","Embarked_Code"]
dataset = data.drop(columns=drop_columns)
train = dataset[:len(df_train)]
test = dataset[len(df_train):]
dataset.columns

Index(['PassengerId', 'Perished', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Fare', 'Age_Code', 'C', 'Q', 'S'],
      dtype='object')

In [7]:
X = train.iloc[:, 2:].values
y = train.iloc[:, 1].values
X_test = test.iloc[:, 2:].values

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [10]:
Search = "x"
bestparam = "x"
CV = "x"
Modelname = "RFC"
base_model = RandomForestClassifier(n_estimators = 10, random_state = SEED)
base_model.fit(X_train, y_train)
print('Train Score: {}'.format(round(base_model.score(X_train, y_train), 4)))
print(' Valid Score: {}'.format(round(base_model.score(X_valid, y_valid), 4)))
STrain = base_model.score(X_train,y_train)
SValid = base_model.score(X_valid,y_valid)

Train Score: 0.9691
 Valid Score: 0.7877


In [16]:
Modelname = "RFC"
model = RandomForestClassifier(random_state=SEED)
CV = 5
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
param_grid = {
    'n_estimators':n_estimators,
    'max_depth':[10],
    'min_samples_split':min_samples_split,
    'min_samples_leaf':min_samples_leaf,
    'max_features':max_features,
}
print(param_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_depth': [10], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'max_features': ['auto', 'sqrt']}


In [15]:
Search = "GS"
model_cv = GridSearchCV(estimator=model,param_grid=param_grid,scoring='accuracy',cv=CV,verbose=2,n_jobs=-1)
model_cv.fit(X_train,y_train)
bestparam = str(model_cv.best_params_)

TypeError: Parameter grid for parameter 'max_depth' needs to be a list or a numpy array, but got 10 (of type int) instead. Single values need to be wrapped in a list with one element.

In [ ]:
Search = "RS"
model_cv = RandomizedSearchCV(estimator=model,param_distributions=param_grid,n_iter=100,cv=CV,verbose=1,random_state=SEED,n_jobs=-1)
model_cv.fit(X_train,y_train)
bestparam = str(model_cv.best_params_)

In [17]:
Search = "BS"
model_cv= BayesSearchCV(estimator=model,search_spaces=param_grid,n_iter=100,cv=CV,verbose =2)
model_cv.fit(X_train,y_train)
bestparam = str(model_cv.best_params_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=400; total time=   0.5s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=400; total time=   0.5s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=400; total time=   0.5s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=400; total time=   0.4s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=400; total time=   0.4s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time=   0.9s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time=   0.9s
[CV] END max_depth=10, max_features=s

In [18]:
best_model = model_cv.best_estimator_
print(best_model)
bestparam = str(model_cv.best_params_)
STrain = best_model.score(X_train, y_train)
SValid  = best_model.score(X_valid, y_valid)
print('Train Score: {}'.format(STrain))
print(' Valid Score: {}'.format(SValid))

RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=1600,
                       random_state=42)
Train Score: 0.9171348314606742
 Valid Score: 0.8379888268156425


In [19]:
log = pd.read_csv(PATH_LOG)
Row  =  len(log)
log.loc[Row] = "x"
log.loc[Row,"Modelname"] = Modelname
log.loc[Row,"Search"] = Search
log.loc[Row,"Vestparams"] = bestparam
log.loc[Row,"CV"] = CV
log.loc[Row,"STrain"] = STrain
log.loc[Row,"SValid"] = SValid
log.loc[Row,"Age_Code_N"] = 3
use_columns = test.columns.drop(["PassengerId","Perished"])
for col in use_columns:
    log.loc[Row,col] = "o"
log.fillna("x",inplace=True)
# if PublicScore is knowed
#log.loc[Row,"SPublic"] = 0.791

In [20]:
log.to_csv(PATH_LOG,index_label=False)
log.tail()

,Modelname,STrain,SValid,Pclass,Sex,Age,SibSp,Parch,Fare,C,...,Title_Code,Embarked_Code,Search,CV,Bestparams,Title,FareBin,Fare_Code,Vestparams,Family
54,ETC,0.9831460674157303,0.7821229050279329,o,o,o,o,o,o,x,...,o,o,x,x,x,x,x,x,x,o
55,ETC,0.9719101123595506,0.7877094972067039,o,o,o,o,o,x,x,...,o,o,x,x,x,x,x,o,x,o
56,ETC,0.8567415730337079,0.8212290502793296,o,o,o,o,o,o,x,...,o,o,BS,5,x,x,x,o,"OrderedDict([('max_depth', 40), ('max_features...",o
57,RFC,0.9691011235955056,0.7877094972067039,o,o,o,o,o,o,o,...,x,x,x,5,x,x,x,x,x,x
58,RFC,0.917135,0.837989,o,o,o,o,o,o,o,...,x,x,BS,5,x,x,x,x,"OrderedDict([('max_depth', 10), ('max_features...",x
